In [8]:
import numpy as np
import pandas as pd
from pathlib import Path
import os
import pickle
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [198]:
mod_path="/home/harshit/Project/car_models/2013/"
lis = pd.DataFrame(os.listdir(mod_path))
coeff = pd.DataFrame(columns = ["Symbol","const","SMB %", "HML %","Rm-Rf %"] )
#Making a dataframe of all coffecients
for i in lis[0]:
    data = open(mod_path+i,"rb")
    symbol = i.split('.')[0]
    model = pickle.load(data)
    frame = pd.DataFrame(model.params).T
    frame.insert(0,"Symbol",symbol)
    coeff = coeff.append(frame,ignore_index=True)

/home/harshit/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [199]:
# Making dataframe of Mean Returns
ret_path = "/home/harshit/Project/risk_free_ret/2013/"
lis = pd.DataFrame(os.listdir(ret_path))
returns = pd.DataFrame(columns = ["Symbol","mean_ret"])
factors = pd.read_csv("/home/harshit/Project/allfactors.csv")
for i in lis[0]:
    data = pd.read_csv(ret_path+i)
    data_factors = pd.merge(data,factors,on="Date")
    symbol = i.split(".")[0]
    mean = data_factors.Return.mean()
    df = pd.DataFrame([[symbol,mean]],columns = ["Symbol","mean_ret"])
    returns = returns.append(df,ignore_index=True)
    factor =data_factors[["SMB %","HML %","WML %","Rm-Rf %"]]
    factors_mean = pd.DataFrame(factor.mean(),columns = ["Mean"])


In [200]:
all_data = pd.merge(returns,coeff,on="Symbol", left_index=True, right_index=True) #dataframe of all mean returns and coeff

In [171]:
X = all_data[["const","SMB %","HML %","WML %","Rm-Rf %","high-low"]]
y = all_data[["mean_ret"]]

model1 = sm.OLS(y,X).fit()

KeyError: "['high-low'] not in index"

In [55]:
X = all_data[["const","Rm-Rf %"]]
y = all_data[["mean_ret"]]

model2 = sm.OLS(y,X).fit()

In [56]:
X = all_data[["const","high-low"]]
y = all_data[["mean_ret"]]

model3 = sm.OLS(y,X).fit()

In [201]:
X = all_data[["SMB %","HML %","WML %","Rm-Rf %"]]
y = all_data[["mean_ret"]]

model4 = sm.OLS(y,X).fit()

In [202]:
model_values = pd.DataFrame(model4.params,columns=["Model"]) #comparing means to regression coeffecients
factors_mean.rename(columns={'Mean':"Actual"}, inplace=True)
comparison=pd.merge(model_values,factors_mean,left_index=True,right_index=True)
comparison["%diff"]=(comparison["Actual"]-comparison["Model"])*100/comparison["Actual"]
#comparison["t-stat"]=abs((comparison["Actual"]-comparison["Model"])*np.sqrt(5)/comparison["Model"].std())
#this formula is wrong, we have only one value of t-stat per variable and not YoY values, correct formula will have
#mean of annual values in the numerator
comparison.to_csv("/home/harshit/Project/fama_mcbeth/"+"2013.csv")

In [203]:
comparison

,Model,Actual,%diff
SMB %,0.042250,-0.083088,150.849701
HML %,-0.056243,0.056547,199.462870
WML %,-0.035815,0.109833,132.608516
Rm-Rf %,-0.009123,-0.023308,60.859297


In [205]:
a = pd.read_csv("/home/harshit/Project/fama_mcbeth/2013.csv",index_col=0).drop(["%diff"],axis=1)
a_model=pd.DataFrame(a["Model"]).transpose()
a_act=pd.DataFrame(a["Actual"]).transpose()
b = pd.read_csv("/home/harshit/Project/fama_mcbeth/2014.csv",index_col=0).drop(["%diff"],axis=1)
b_model = pd.DataFrame(b["Model"]).transpose()
b_act=pd.DataFrame(b["Actual"]).transpose()
c = pd.read_csv("/home/harshit/Project/fama_mcbeth/2015.csv",index_col=0).drop(["%diff"],axis=1)
c_model = pd.DataFrame(c["Model"]).transpose()
c_act=pd.DataFrame(c["Actual"]).transpose()
d = pd.read_csv("/home/harshit/Project/fama_mcbeth/2016.csv",index_col=0).drop(["%diff"],axis=1)
d_model=pd.DataFrame(d["Model"]).transpose()
d_act=pd.DataFrame(d["Actual"]).transpose()
e = pd.read_csv("/home/harshit/Project/fama_mcbeth/2017.csv",index_col=0).drop(["%diff"],axis=1)
e_model= pd.DataFrame(e["Model"]).transpose()
e_act=pd.DataFrame(e["Actual"]).transpose()
model = a_model.append(b_model).append(c_model).append(d_model).append(e_model)
actual= a_act.append(b_act).append(c_act).append(d_act).append(e_act)
#pd.DataFrame(all_years.abs().mean(axis=1),columns=["Mean t-stat"])

In [207]:
t-stat=abs((pd.DataFrame(np.mean(actual,axis=0))-pd.DataFrame(np.mean(model,axis=0)))*np.sqrt(5)/comparison["Model"].std())

SyntaxError: can't assign to operator (<ipython-input-207-88707318f642>, line 1)

In [208]:
abs(pd.DataFrame(np.mean(actual,axis=0))- pd.DataFrame(np.mean(model,axis=0)))*np.sqrt(5)/pd.DataFrame(model.std())

,0
SMB %,0.410164
HML %,0.709808
WML %,0.895592
Rm-Rf %,2.151155


In [157]:
pd.DataFrame(model.std())

,0
SMB %,0.072209
HML %,0.134994
WML %,0.072226
Rm-Rf %,0.086815
high-low,0.271619
